# Assignment 4: Lab Exercise

We'll study and implement two quantum algorithms through this assignment

---

## **Bernstein-Vazirani Algorithm**


#### Problem Statement:

- Given an oracle function $f:\{0,1\}^n \rightarrow \{0,1\}$, defined as $ f(\mathbf{x}) = x \cdot s $, find the secret bit string $s$.

**Note:**

- $x \cdot s$ represents the inner (dot) product of the bit strings modulo $2$.

- The size of the bitstring '$s$' should be the same as the size of input $x$ for the operation $x.s$ to make sense.

- The unitary operator representing the oracle now takes the form:  
$$ U_f: \ket{x}\ket{y} \mapsto \ket{x}\ket{y \oplus ( x \cdot s)} $$


#### <font color='red'>Exercise:</font>

Find $s$ for the function that provides the below outputs: (here $n=2$)
\begin{align*}
	f(00)&=0\\
	f(01)&=1\\
	f(10)&=1\\
	f(11)&=0\\
\end{align*}

Example: Say $x=10$, from above $f(10)= 1= x.s$
$$ 1= x.s= (x_1x_2).(s_1s_2)= (x_1s_1 + x_2s_2 ) mod 2= 1s_1 + 0s_2 (mod 2)= s_1$$
Thus $s_1= 1$, similarly find $s_2$ to get the full string $s_1s_2$.

Note that for this function you would require $2$ calls to the function to find $s$ completely.

      (your answer here)

#### <font color='red'> **Exercise:** *Classical Complexity-* Clasically, how many oracle calls you need to make to find $s$ for  $f:\{0,1\}^n \rightarrow \{0,1\}$, defined as $ f(\mathbf{x}) = x \cdot s $ ? </font>

        (your answer here)

---

#### **Quantum Algorithm:** 

We construct a $n+1$ qubit quantum circuit.

- Set the ($n+1$)th qubit to state $\ket{-}$ by applying $X$ and $H$ gates.
- Apply $H$ to first $n$ qubits.
- Apply $U_f$. (*the oracle*: required to be given to the algorithm)
- Apply $H$ to first $n$ qubits.
- Measure the first $n$ qubits to obtain $s$.

This is exactly the same algorithm as the Deutsch-Jozsa which we covered in a previous lab material. Only the oracle will have a slighly modified form. 

![DeutschjoZSACircuit](Images/DJBV.png)

#### <font color='red'>Exercise: Work out the stages of the algorithm looking at the circuit above and check whether you get the below intermediate states: </font>

$$ \ket{\psi_0} = \ket{0}^{\otimes n} \ket{0}$$

$$ \ket{\psi_1} = \ket{0}^{\otimes n} \ket{1}$$

$$ \ket{\psi_2} = \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1} \ket{x} \otimes \ket{-} $$

$$ \ket{\psi_3} = \bigg[ \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{x.s} \ket{x} \bigg] \otimes \ket{-} $$

        (your answer here)


- Remember from the $PS(07)$ notebook, $ H^{\otimes n} \ket{x} = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{x \cdot z} \ket{z}. $

- We also know that the $H^{\otimes n}$ operator is its own inverse. Thus, $H^{\otimes n}\ket{a} = \ket{b} \Longleftrightarrow H^{\otimes n}\ket{b} = \ket{a}$. 

So, the first $n$ qubit state of $\ket{\psi_3}$ is exactly the state obtained after applying $H^{\otimes n}$ to $\ket{s}$.


Now applying $H^{\otimes n}$ to the input qubits (the first $n$ qubits), should provide us: (due to the idea discussed in the two bullet points above)

$$ \ket{\psi_4} = \ket{s} \otimes \ket{-} $$

**Measurement of the first $n$ qubits will provide string $s$ with probability $1$.**

*Note that the Bernstein-Vazirani problem was artificially created to demonstrate the advantage of a quantum algorithm over classical algorithms.*

 To implement the algorithm, we will first design some specific oracles and then use the algorithm to determine the string $s$ encoded in the oracle.

#### <font color= 'red'> Exercise: Design the oracles given below </font> 
(here n=4 , hence we would require a Quantum Circuit with 5 qubits)

- Given $\textbf{s} = 0110$, implement a function that returns an oracle for the function  $ f(\mathbf{x}) = \mathbf{x} \cdot \mathbf{s} $. *(oracle1, has been implemented for you below)*


- Given $\textbf{s} = 1001$, implement a function that returns an oracle for the function  $ f(\mathbf{x}) = \mathbf{x} \cdot \mathbf{s} $. *(oracle2, you need to do implement)*

In [ ]:
import qiskit
from qiskit import QuantumCircuit

def oracle1():  #for s=0110
    circuit = QuantumCircuit(5)
    circuit.barrier()

    circuit.cx(1, 4)
    circuit.cx(2, 4)

    circuit.barrier()
    return circuit


def oracle2(): #for s= 1001
    circuit = QuantumCircuit(5)
    circuit.barrier()

    #your code here

    circuit.barrier()
    return circuit

#### <font color= 'red'> Exercise: </font> 
From the implementation above, how do you generalize the oracle construction for any bitstring $s$, irrespective of its size.
(you can either provide a function generalizing the oracle construction for any $s$ or you can write the implementation logic in words)

        (your answer here)

In [ ]:
#Chosing a random oracle for using it to test the BV algorithm implementation

import random
# Create the list of oracles
oracle_list = [oracle1, oracle2]

# Randomly select an index using randrange(4)
random_index = random.randrange(2)

print(random_index)

# Call the selected oracle
selected_oracle = oracle_list[random_index]

selected_oracle()

##### Given the oracle function `selected_oracle()` for $f$, we construct a circuit that implements the Bernstein-Vazirani algorithm described above to find out $s$.

*(the algorithm has been implemented for you, just look at the steps and check if it provides the desired string)*

In [ ]:
from qiskit import QuantumCircuit, execute, Aer

n=4 #(size of the string or the input 'x')

#Initialization
bv_circuit = QuantumCircuit(n+1, n)  #measurement only on the first n qubits, hence n classical registers would suffice
bv_circuit.x(n)
bv_circuit.h(range(n+1))

#Apply oracle
bv_circuit.compose(selected_oracle(), inplace=True)

#Apply Hadamard to all qubits and measure the first n qubits
bv_circuit.h(range(n))
bv_circuit.measure(range(n), range(n))

#Draw the circuit, if needed

#bv_circuit.draw(output="mpl")

#extracting the results of measurement to determine 's'
job = execute(bv_circuit, Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts()
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome,"is observed",counts[outcome],"times")

Since a classical computation of $s$ requires __ function calls, we have obtained a “quantum speedup” of $n$ (a **polynomial speedup**). 

The procedure is analagous to Deutsch-Jozsa algorithm. The first set of Hadamards generates a superposition of inputs to the oracle $U_f$ which evaluates the function for all $2^{n}$ inputs (*quantum parallelism*), and then the second set of Hadamards destroys all the outputs apart from $s$, using *quantum interference*.


---

#### <font color= 'red'> Exercise: </font> 

- Given $\textbf{s} = 0110101$, implement a function that returns an oracle for the function  $ f(\mathbf{x}) = \mathbf{x} \cdot \mathbf{s} $.
- Implement the Berstein-Vazirani algorithm for the above oracle and check if you obtain the required string.
- What is the circuit width and depth used for solving this specific problem?


In [ ]:
#your codes here (write all in a single cell)

        (your answer here)

---

## **Simon's Algorithm**

#### Problem Statement:

- Given an oracle function $f: \{0,1\}^{n} \rightarrow \{0,1\}^{n}$, such that $f(x) = f(x\oplus s)$, the goal is to determine the bitstring $s$ (where $s$ $\in$ {0,1}$^{n}$ and $s\ne 0^{n}$).


**Note:**

- Unlike the previous problem, here we don't know the functional form of $f$.
- The secret string $s$ can be considered as a "mask" which hides the underlying encoding ($f$).
- **As long as $s$ is not the zero bitstring, the function is two-to-one i.e. mapping two elements from the domain to one value from the range.**

<i><font color= 'yellow'>**Simon's problem can also be defined as the problem of determining whether $f$ is two-to-one or one-to-one, in which case one needs to determine whether $s$ is a zero string or not.</i>**</font>

#### <font color= 'red'> Exercise: </font> 

- For two inputs $x_1$ and $x_2$ for which $f(x_1) = f(x_2)$, show that $x_1⊕x_2$ yields the secret string $s$, given that the function $f$ satisfies the property $f(x)=f(x\oplus s)$.
- How do you approach the Simon's problem classically? What is the classical complexity for the Simon's problem? (look at the worst-case scenario)

        (your answer here)

---

##### **Optional**
 
Have you heard of the Birthday problem?

- In a set of $n$ randomly chosen people, what should be the value of $n$ such that the probability of at least two people sharing a birthday exceeds 50%?

You can read more about the Birthday problem [here](https://betterexplained.com/articles/understanding-the-birthday-paradox/). Can you relate it to the classical way of solving the Simon's problem above?

---

#### **Quantum Algorithm:**



We construct a circuit with two sets of quantum registers, each register contains $n$ qubits <br>

*   Apply $H$ gates to all the qubits in the **first** register set
*   Apply the oracle function to the full circuit
*   Measure the second register
*   Apply $H$ gates to all the qubits in the first register
*   Measure the first register and record the $n$-bit string as your sample for $y$
*   Repeat the above steps until you get $n-1$ distinct strings $y$
*   Construct a set of linear equations using the bitstrings $y$ and solve for $s$

![SimonCircuit](Images/SimonCirc.png)

#### Let's work out the stages of the algorithm looking at the circuit above:

The oracle is defined as follows:
$$ U_f(\ket{x}\ket{y}) = \ket{x} \ket{f(x) \oplus y}$$

From the figure,

- Start with the initial state |0⟩$^{\otimes n}$ in both registers. We need two sets of registers each with $n$ qubits, where $n$ represents the number of input bits $f$ takes:

$$ |{\psi_1}⟩ = |{0}⟩^{\otimes n} |{0}⟩^{\otimes n}$$

- Hadamard to first register set results in:
$$ |{\psi_2}⟩ = \frac{1}{\sqrt{2^{n}}}\sum_{x ∈ \left\{0,1\right\}^{n}}|x⟩ |0⟩^{\otimes n} $$

- Action of oracle function $U_{f}$. As the output of $U_{f}$, the first register remains unchanged, but the second register will have $\ket{f(x) \oplus 0}$  (as per the definition of $U_f$ given above)

$$ |{\psi_3}⟩ = \frac{1}{\sqrt{2^{n}}}\sum_{x ∈ \left\{0,1\right\}^{n}}|x⟩ |𝑓(x)⟩ $$

- Measure the second register to randomly observe one of the output values. Say the observed outcome is $f(z)$ for some $z \in \{0,1\}^n$.

---

#### **Case 1:** 

$f(x)$ is a **two-to-one function** (i.e. $s \neq 0^n$), we get a superposition of the two input values $\ket{z}$ and $\ket{z \oplus s}$ that can produce the observed output $f(z)$ in the second register: Thus, 

$$ |{\psi_4}⟩ = \frac{1}{\sqrt{2}}(\ket{z}+\ket{z\oplus s})\ket{f(z)} $$

##### Recalling, $ H^{\otimes n}\ket{z} =  \frac{1}{\sqrt{2^n}} \sum_{y \in \{0,1\}^n} (-1)^{z \cdot y}\ket{y} $

- Applying Hadamard transform only to the **first register**.

$$
|{\psi_5}⟩ = \frac{1}{\sqrt{2^{n+1}}}\left[\sum_{y ∈ \left\{0,1\right\}^{n}}[(-1)^{z \cdot y}+(-1)^{(z\oplus s) \cdot y}]\ket{y}\right] \ket{f(z)}  
$$

- We now measure the first register. 

---

##### <font color= 'red'> Exercise: </font> Check that when $(-1)^{z \cdot y} \neq (-1)^{(z\oplus s) \cdot y}$, the term $(-1)^{z \cdot y}+(-1)^{(z\oplus s) \cdot y}$ is 0, and the probability of observing such $y$ is equal to 0.

        (your answer here)
---

So for some $\ket{y}$ to have nonzero probability of measurement (as some $y$ is definitely measured), the condition $ (-1)^{z \cdot y} = (-1)^{(z\oplus s) \cdot y}$ should be satisfied.

This implies, $${z \cdot y} = {(z\oplus s) \cdot y}$$

##### <font color= 'red'> Exercise: </font> Show that the above condition leads to $s \cdot y = 0$.

        (your answer here)

---

Now $s\cdot y=0$,  which implies $(-1)^{z \cdot y}+(-1)^{(z\oplus s) \cdot y} = \pm 2$ and we have a factor of $2$ inside in the expression for $\ket{\psi_5}$. Thus, probability of observing $ y $ is  

$$\left (\frac{-2}{\sqrt{2^{n+1}}} \right )^2 = \frac{1}{2^{n-1}}$$ 
(which is a constant as $n$ is fixed)

Hence any string $y$ that satisfies the property $s\cdot y=0$ will be observed with equal probability and $y$ with the property $s\cdot y=1 $ will not be observed. 

**From case $1$ we deduced that, we will always obtain a $y$ for which $s\cdot y=0$.** 



---

#### **Case 2:**

If $s = 0^n$, i.e $f(x)$ is a **one-to-one function**, then we obtain, 

$$ \ket{{\psi_4}} =\ket{z}\ket{f(z)} $$

- Applying Hadamard transform only to the **first register** we get:
$$
\ket{{\psi_4}} = \frac{1}{\sqrt{2^{n}}}\sum_{y ∈ \left\{0,1\right\}^{n}}(-1)^{z \cdot y}\ket{y}
$$
Hence any one of the states $\ket{y}$ is observed with **equal probability.**

Since $s = 0^n$, $s.y=0$ for all $y$. 

**In this case by default $s.y=0$ for all $y$ observed**

---

#### Post-Processing
*(this refers to extracting the required results from the quantum circuit results)*

Irrespective of which case we get, we found out a string $ y $ that always satisfies $ s \cdot y=0 $. 

Now, we make $n-1$ queries to the oracle. We get $n-1$ **linearly independent** strings \{$y^1,...,y^{n-1}$\} so that we get a system of $n-1$ equations in $n$ unknowns in modulo $2$ satisfying,

$ y^1 \cdot s=0, y^2 \cdot s=0, y^3 \cdot s=0 ,..., y^{n-1} \cdot s=0 $

---

##### **Optional Read 2:**

But some of the strings obtained could be same (thus linearly dependent). We can do a simple probability exercise,

The probability that $n-1$ linearly independent strings are obtained in $n-1$ runs (or queries) is at least

$$
\prod_{k=1}^{\infty}1-\left (\frac{1}{2} \right )^k = 0.288888... > 1/4
$$

So if you repeat whole process $4m$ times then the probability $P_M$ of finding a linearly dependent set:

$$
P_M < (1-(1/4))^{4m} < \epsilon.
$$

Hence for any $\epsilon > 0$, by an appropriate choice of $m$ Simon's Algorithm can solve the problem making $O(n)$ queries with error probability of at most $\epsilon$.

---

##### <font color= 'red'> Exercise: </font>

The oracle function `simon_oracle()` which implements $f(x)=f(x \oplus s)$ where $s$ is a 3-bit string, is given to you. Complete the function `simon` that implements the Simon's Algorithm. Note that each run of Simon's algorithm returns a single bitstring, $y$.

You are also provided with the code which calls `simon` until $n-1$ linearly independent bistrings $y$ are obtained.

In [ ]:
## This oracle function was taken as it is from qworld resources. 

def simon_oracle():
    qreg1 = QuantumRegister(3, "register_1")
    qreg2 = QuantumRegister(3, "register_2")
    creg = ClassicalRegister(3)

    simon_circuit = QuantumCircuit(qreg1, qreg2, creg)

    #map 000 and 010 to 000
    #Do nothing

    #map 111 to 100
    simon_circuit.mcx([0,1,2],5) 
    simon_circuit.barrier()

    #map 101 to 100
    simon_circuit.x(1)
    simon_circuit.mcx([0,1,2],5)
    simon_circuit.x(1)
    simon_circuit.barrier()

    #map 110 to 110
    simon_circuit.x(0)
    simon_circuit.mcx([0,1,2],4)
    simon_circuit.mcx([0,1,2],5)
    simon_circuit.x(0)
    simon_circuit.barrier()

    #map 100 to 110
    simon_circuit.x(0)
    simon_circuit.x(1)
    simon_circuit.mcx([0,1,2],4)
    simon_circuit.mcx([0,1,2],5)
    simon_circuit.x(0)
    simon_circuit.x(1)
    simon_circuit.barrier()    
    
    #map 001 to 010
    simon_circuit.x(1)
    simon_circuit.x(2)
    simon_circuit.mcx([0,1,2],4)
    simon_circuit.x(1)
    simon_circuit.x(2)
    simon_circuit.barrier()
    
    #map 011 to 010
    simon_circuit.x(2)
    simon_circuit.mcx([0,1,2],4)
    simon_circuit.x(2)
    return simon_circuit


In [ ]:
from qiskit import QuantumCircuit, execute, Aer, QuantumRegister, ClassicalRegister

#Implementing Simon's algorithm 

def simon(n):  #n is the length of 's' (or the length of input to the function 'x')
    # Define the set of registers
    qreg1 = QuantumRegister(n, "register_1")
    qreg2 = QuantumRegister(n, "register_2")
    creg = ClassicalRegister(n)

    # Define the quantum Circuit
    simon_circuit = QuantumCircuit(qreg1, qreg2, creg)

    # Apply Hadamard to the first set of register


    # Apply the simon_oracle function


    # Apply Hadamard to the first set register


    # Measure the first register


    job = execute(simon_circuit, Aer.get_backend('qasm_simulator'),shots=1)
    counts = job.result().get_counts() #Note that counts is a dictionary

    #Since the number of shots is 1, there is only a single key and we return it
    return list(counts.keys())[0]

# n=    (chose n appropriately and uncomment)
y_list = []
iter = 1
while(len(y_list)) < n-1:  # while loop till we get (n-1) linearly independent strings
    y = simon(n)
    if y!='000' and y not in y_list: #Omit 000 string and omit if y already exists in the list
        y_list.append(y) #we get the key and append it to our list
    iter+=1
print(y_list)
print(f"{iter} iterations were required to obtain the list containing (n-1) linearly independent y's") 

**Post-Processing:**


After obtaining the list of bitstrings $y$ from the computation above, find the hidden bitstring $s$ from the output by solving the linear systems of equations ($s.y=0$)

When solving for $s$, you'll get two solutions. One solution is $s$ = |0⟩$^{\otimes n}$ which trivially satisfies $s.y=0$, discard this solution and keep the other one as the provided $f$ is **two-to-one** and has a non-trivial $s$.

        (your answer here)

---

### References


- QWorld resources available at https://gitlab.com/qworld/qeducation/qbook101
- Nielsen, M. A., & Chuang, I. L. (2010). Quantum Computation and Quantum Information: 10th Anniversary Edition. Cambridge: Cambridge University Press.
- Qiskit API reference found at https://docs.quantum.ibm.com/api/qiskit
- Birthday Paradox reference: https://betterexplained.com/articles/understanding-the-birthday-paradox/